# 解谜游戏：Number Link（数链？）

- 用一条连续的线把相同的数字连接起来；
- 线只能从上下左右四个方向穿过图案，不能两次经过同一个格子；
- 线不能分支、不能交叉、不能经过数字格；


----------------

- Connect pairs of the same numbers with a continuous line.
- Lines go through the center of the cells, horizontally, vertically, or changing direction, and never twice through the same cell.
- Lines cannot cross, branch off, or go through the cells with numbers.

-------

REF: [Link](https://github.com/uguryavuz/numberlink-solver)

In [1]:
from ortools.sat.python import cp_model as cp

def numberlink_solver(grid_ipt , X, Y):
    
    grid = grid_ipt.split(",")
    try:
        len(grid) == X * Y
    except Exception:
        raise(f"检查输入棋盘的长度： length of grid != {X} * {Y}")

    ods = max(map(int, grid))
    model = cp.CpModel()
    x = {}
    y = {}
    # 表示每个格子属于第几个线路

    for i in range(Y):
        for j in range(X):
            if grid[i * X + j] == "0":
                y[i, j] = model.NewIntVar(1, ods, name = f"y[{i}, {j}]")
            else:
                y[i, j] = int(grid[i * X + j])
            
            if i + 1 < Y:
                x[i, j , 1, 0] = model.NewBoolVar(name = f"x[{i}, {j}, 1, 0]")
            else:
                x[i, j , 1, 0] = 0
            
            if i - 1 >= 0:
                x[i, j , -1, 0] = model.NewBoolVar(name = f"x[{i}, {j}, -1, 0]")
            else:
                x[i, j , -1, 0] = 0
                
            if j + 1 < X:
                x[i, j , 0, 1] = model.NewBoolVar(name = f"x[{i}, {j}, 0, 1]")
            else:
                x[i, j , 0, 1] = 0
            
            if j - 1 >= 0:
                x[i, j , 0, -1] = model.NewBoolVar(name = f"x[{i}, {j}, 0, -1]")
            else:
                x[i, j , 0, -1] = 0
            
            if grid[i * X + j] != "0":
                model.Add(x[i, j , 1, 0] + x[i, j , -1, 0] + x[i, j ,0 , -1] + x[i, j , 0, 1] == 1)
            else:
                model.Add(x[i, j , 1, 0] + x[i, j , -1, 0] + x[i, j ,0 , -1] + x[i, j , 0, 1] == 2)
    
    for i in range(Y):
        for j in range(X):
            if j - 1 >= 0:
                model.Add(y[i, j] == y[i, j - 1]).OnlyEnforceIf(x[i, j , 0, -1])
            if j + 1 < X:
                model.Add(y[i, j] == y[i, j + 1]).OnlyEnforceIf(x[i, j , 0,  1])
            if i + 1 < Y:
                model.Add(y[i, j] == y[i + 1, j]).OnlyEnforceIf(x[i, j , 1, 0])
            if i - 1 >= 0:
                model.Add(y[i, j] == y[i - 1, j]).OnlyEnforceIf(x[i, j , -1, 0])

    
    solver = cp.CpSolver()
    status = solver.Solve(model)

    if status == cp.OPTIMAL:
        for i in range(Y):
            for j in range(X):
                print(solver.Value(y[i, j]), end = " ")
            print()
        print()
        print("NumConflicts:", solver.NumConflicts())
        print("NumBranches:", solver.NumBranches())
        print("WallTime:", solver.WallTime())

    else:
        print("Unable to find the OPTIMAL.")

if __name__ == "__main__":
    grid = "0,11,4,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,0,0,0,2,0,0,0,0,9,0,0,1,0,11,0,0,3,0,10,9,0,3,0,0,0,0,0,0,0,0,0,0,0,7,0,0,7,0,0,0,0,0,0,0,0,0,5,0,0,8,6,0,0,0,6,0,0,0,8,1,0,0,4,0,0,0,0,0,0,0,0,5"
    numberlink_solver(grid, 10, 10)

11 11 4 3 3 3 1 1 1 1 
11 2 4 3 3 3 1 1 1 1 
11 2 4 3 3 10 10 10 1 1 
11 2 4 3 3 10 9 9 1 1 
11 11 4 3 3 10 10 9 3 3 
4 4 4 7 7 1 1 1 3 3 
4 7 7 7 7 1 1 1 6 6 
4 6 6 5 5 1 8 8 6 6 
4 6 6 5 5 1 8 1 5 5 
4 6 6 5 5 1 1 1 5 5 

NumConflicts: 0
NumBranches: 664
WallTime: 0.030032000000000003



![](../assets/figures/NumberLink.png)

![](../assets/figures/NumberLink1.png)